# SUPERMARKET SALES

Proyecto de análisis de Series Temporales y Forecast
-- Análisis Serie Temporal - Forecast --

Jose Luis Quintero García, abr2023

## Importación de librerías y carga de datos

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [88]:
ordenes = pd.read_csv("./ordenes.csv", parse_dates=["Order Date", "Ship Date"])
ordenes.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,City,Product ID,Sales,Quantity,Discount,Profit,Devolucion
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Henderson,FUR-BO-10001798,261.9600,2,0.00,41.9136,0
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Henderson,FUR-CH-10000454,731.9400,3,0.00,219.5820,0
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Los Angeles,OFF-LA-10000240,14.6200,2,0.00,6.8714,0
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Fort Lauderdale,FUR-TA-10000577,957.5775,5,0.45,-383.0310,0
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Fort Lauderdale,OFF-ST-10000760,22.3680,2,0.20,2.5164,0


In [89]:
# Quitamos los registros donde haya una devolución
ordenes = ordenes.loc[ordenes["Devolucion"]==0]

# Borramos la columna "devolucion", que ahora sobra
ordenes.drop(["Devolucion"], axis=1, inplace=True)

## Análisis

El trabajo a realizar es el siguiente:

1. Expondremos los datos de ventas en la ciudad de New York (la que mayores ventas presenta y la más rentable) y haremos un análisis de la serie temporal resultante, con datos de ventas por días.
2. Analizaremos si la serie es o no estacionaria y si presenta algún tipo de patrón de estacionalidad. 
3. Con ello determinaremos el mejor modelo para realizar un forecast de ventas durante el próximo mes.

In [90]:
analisis = ordenes.loc[ordenes["City"] == "New York City"]
analisis.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,City,Product ID,Sales,Quantity,Discount,Profit
53,CA-2016-105816,2016-12-11,2016-12-17,Standard Class,JM-15265,New York City,OFF-FA-10000304,15.260,7,0.0,6.2566
54,CA-2016-105816,2016-12-11,2016-12-17,Standard Class,JM-15265,New York City,TEC-PH-10002447,1029.950,5,0.0,298.6855
70,CA-2016-106075,2016-09-18,2016-09-23,Standard Class,HM-14980,New York City,OFF-BI-10004654,4.616,1,0.2,1.7310
96,CA-2017-161018,2017-11-09,2017-11-11,Second Class,PN-18775,New York City,FUR-FU-10000629,96.530,7,0.0,40.5426
110,CA-2017-146780,2017-12-25,2017-12-30,Standard Class,CV-12805,New York City,FUR-FU-10001934,41.960,2,0.0,10.9096


In [91]:
analisis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 870 entries, 53 to 9940
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Order ID     870 non-null    object        
 1   Order Date   870 non-null    datetime64[ns]
 2   Ship Date    870 non-null    datetime64[ns]
 3   Ship Mode    870 non-null    object        
 4   Customer ID  870 non-null    object        
 5   City         870 non-null    object        
 6   Product ID   870 non-null    object        
 7   Sales        870 non-null    float64       
 8   Quantity     870 non-null    int64         
 9   Discount     870 non-null    float64       
 10  Profit       870 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 81.6+ KB


Vemos que tenemos 870 registros. SIn embargo, no son continuos, pues no todos los días hay pedidos de New York. Rellenamos esos días faltantes con 0

In [92]:
analisis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 870 entries, 53 to 9940
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Order ID     870 non-null    object        
 1   Order Date   870 non-null    datetime64[ns]
 2   Ship Date    870 non-null    datetime64[ns]
 3   Ship Mode    870 non-null    object        
 4   Customer ID  870 non-null    object        
 5   City         870 non-null    object        
 6   Product ID   870 non-null    object        
 7   Sales        870 non-null    float64       
 8   Quantity     870 non-null    int64         
 9   Discount     870 non-null    float64       
 10  Profit       870 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 81.6+ KB


In [95]:
# Vamos a agrupar los datyos por días. Aunque perderemos las variables categóricas, no nos importa, porque lo que queremos analizar aquí
# son los datos de ventas o beneficio
analisis = analisis \
    .groupby("Order Date") \
    .aggregate({'Order ID': "count", \
                'Sales': "sum", \
                'Quantity': "sum", \
                'Profit': "sum" }) \
    .sort_values("Order Date", ascending=True)  

# Dado que en esta agrupación hay días sin ventas, aparecen gaps en la serie. Vamos a corregir esto:
analisis = analisis.resample("D").asfreq()   # El valor "D" nos agrupa valores a dias
analisis = analisis.fillna(0)
analisis

,Order ID,Sales,Quantity,Profit
Order Date,,,,
2014-01-28,1,3.928,1.0,1.3257
2014-01-29,1,0.000,0.0,0.0000
2014-01-30,1,0.000,0.0,0.0000
2014-01-31,1,0.000,0.0,0.0000
2014-02-01,1,0.000,0.0,0.0000
...,...,...,...,...
2017-12-26,1,0.000,0.0,0.0000
2017-12-27,1,0.000,0.0,0.0000
2017-12-28,1,134.524,21.0,56.4337
